# Window Functions com DuckDB

* [Window Functions](https://duckdb.org/docs/stable/sql/functions/window_functions)

In [1]:
import duckdb

In [2]:
conn = duckdb.connect()

In [3]:
conn.execute("""
    CREATE OR REPLACE TABLE vendas (
        id INTEGER,
        vendedor TEXT,
        regiao TEXT,
        valor NUMERIC,
        data DATE
    );
""")

In [4]:
dados = [
    (1, 'Ana',    'Sul',   1000, '2024-01-01'),
    (2, 'Bruno',  'Sul',   1500, '2024-01-02'),
    (3, 'Carla',  'Norte', 1200, '2024-01-03'),
    (4, 'Ana',    'Sul',   900,  '2024-01-04'),
    (5, 'Bruno',  'Sul',   2000, '2024-01-05'),
    (6, 'Carla',  'Norte', 1100, '2024-01-06'),
    (7, 'Ana',    'Sul',   1500, '2024-01-07')
]

* [Prepared Statements](https://duckdb.org/docs/stable/clients/python/dbapi#prepared-statements)

In [5]:
conn.executemany(
    "INSERT INTO vendas VALUES (?, ?, ?, ?, ?);",
    dados
)

* [Aggregate Window Functions](https://duckdb.org/docs/stable/sql/functions/window_functions#aggregate-window-functions)

In [6]:
display(conn.execute("""
    SELECT
        vendedor,
        data,
        valor,
        SUM(valor) OVER (PARTITION BY vendedor ORDER BY data) AS soma_acumulada
    FROM vendas;
""").df())

,vendedor,data,valor,soma_acumulada
0,Bruno,2024-01-02,1500.0,1500.0
1,Bruno,2024-01-05,2000.0,3500.0
2,Carla,2024-01-03,1200.0,1200.0
3,Carla,2024-01-06,1100.0,2300.0
4,Ana,2024-01-01,1000.0,1000.0
5,Ana,2024-01-04,900.0,1900.0
6,Ana,2024-01-07,1500.0,3400.0


* [RANK](https://duckdb.org/docs/stable/sql/functions/window_functions#rankorder-by-ordering)

In [7]:
display(conn.execute("""
    SELECT
        vendedor,
        valor,
        regiao,
        RANK() OVER (PARTITION BY regiao ORDER BY valor DESC) AS ranking
    FROM vendas;
""").df())

,vendedor,valor,regiao,ranking
0,Carla,1200.0,Norte,1
1,Carla,1100.0,Norte,2
2,Bruno,2000.0,Sul,1
3,Bruno,1500.0,Sul,2
4,Ana,1500.0,Sul,2
5,Ana,1000.0,Sul,4
6,Ana,900.0,Sul,5


* [LAG](https://duckdb.org/docs/stable/sql/functions/window_functions#lagexpr-offset-default-order-by-ordering-ignore-nulls)

In [8]:
display(conn.execute("""
    SELECT
        vendedor,
        data,
        valor,
        LAG(valor) OVER (PARTITION BY vendedor ORDER BY data) AS valor_anterior
    FROM vendas;
""").df())

,vendedor,data,valor,valor_anterior
0,Carla,2024-01-03,1200.0,NaN
1,Carla,2024-01-06,1100.0,1200.0
2,Ana,2024-01-01,1000.0,NaN
3,Ana,2024-01-04,900.0,1000.0
4,Ana,2024-01-07,1500.0,900.0
5,Bruno,2024-01-02,1500.0,NaN
6,Bruno,2024-01-05,2000.0,1500.0


* [NTILE](https://duckdb.org/docs/stable/sql/functions/window_functions#ntilenum_buckets-order-by-ordering)

In [9]:
display(conn.execute("""
    SELECT
        vendedor,
        valor,
        NTILE(4) OVER (ORDER BY valor DESC) AS quartil
    FROM vendas;
""").df())

,vendedor,valor,quartil
0,Bruno,2000.0,1
1,Bruno,1500.0,1
2,Ana,1500.0,2
3,Carla,1200.0,2
4,Carla,1100.0,3
5,Ana,1000.0,3
6,Ana,900.0,4


In [10]:
conn.close()